## Importing Libraries

In [1]:
import io

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)
# Make sure you are running this in Python3 environment and TFv2.x

2.2.0


## Importing IMDB Dataset

In [2]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

## Splitting the Data (train-test) and Separating Labels and Sentences

In [3]:
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

## Tokenizing and Padding

In [4]:
# Defining hyper-parameters
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

# Tokenizing and Padding
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

## De-Tokenizing

In [7]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print('----Tokenized----')
print(decode_review(padded[1]),'\n')
print('----Original----')
print(training_sentences[1])

----Tokenized----
b perhaps because i was so young innocent and <OOV> when i saw it this movie was the cause of many <OOV> nights for me i haven't seen it since i was in seventh grade at a <OOV> school so i am not sure what effect it would have on me now however i will say that it left an impression on me and most of my friends it did serve its purpose at least until we were old enough and <OOV> enough to analyze and create our own opinions i was particularly terrified of what the newly converted post rapture christians had to endure when not receiving the mark of the beast i don't want to spoil the movie 

----Original----
b"Perhaps because I was so young, innocent and BRAINWASHED when I saw it, this movie was the cause of many sleepless nights for me. I haven't seen it since I was in seventh grade at a Presbyterian school, so I am not sure what effect it would have on me now. However, I will say that it left an impression on me... and most of my friends. It did serve its purpose, at 

## Building a Model

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


## Fitting

In [9]:
num_epochs = 10
model.fit(
    padded, 
    training_labels_final, 
    epochs=num_epochs, 
    validation_data=(
        testing_padded, 
        testing_labels_final
    )
)

Epoch 1/10
782/782 [==============================] - 3s 4ms/step - loss: 0.5115 - accuracy: 0.7266 - val_loss: 0.3765 - val_accuracy: 0.8286
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2422 - accuracy: 0.9059 - val_loss: 0.3654 - val_accuracy: 0.8413
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0887 - accuracy: 0.9770 - val_loss: 0.4460 - val_accuracy: 0.8296
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0220 - accuracy: 0.9975 - val_loss: 0.5197 - val_accuracy: 0.8312
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0055 - accuracy: 0.9995 - val_loss: 0.6037 - val_accuracy: 0.8230
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.6295 - val_accuracy: 0.8294
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 8.3509e-04 - accuracy: 1.0000 - val_loss: 0.6710 - val_accuracy: 0.8288
Ep

## Checking Volcabulary Size and Embedding Dimensions

In [12]:
e = model.layers[0]
weights = e.get_weights()[0]
print('vocab_size, embedding_dim: ')
print(weights.shape)

vocab_size, embedding_dim: 
(10000, 16)


## Visualising the Vectors and Metadata

In [14]:
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

# Downloading vectors and metadata files
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')

# These files can be visulaised at: http://projector.tensorflow.org/

<img='../data/images/IMDB/vec-meta-tsv.PNG'>

<img src="../data/images/IMDB/vec-meta-tsv.PNG">

## Testing

In [16]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1429], [969], [4], [1537], [1537], [4749], [], [790], [2012], [11], [2920], [2188], [], [790], [2012], [11], [579], [], [11], [579], [], [4], [1783], [4], [4512], [11], [2920], [1277], [], [], [2012], [1003], [2920], [969], [579], [790], []]
